In [15]:
import pandas as pd

In [16]:
import numpy as np

In [17]:
import pickle
import os

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

In [19]:
if(os.path.exists('chkpt.pckl')):
    emodata = pickle.load(open('chkpt.pckl','rb'))
else:
    emodata = pd.read_csv('EmotionCleanTagged.csv')
    emodata['Label'] = emodata[['Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']].apply(lambda x: list((x.dropna().values)), axis=1)
    pickle.dump(emodata,open('chkpt.pckl','wb'))

In [20]:
X_train = np.array(emodata['English Word'])
y_train_text = np.array(emodata['Label'])

In [22]:
if(os.path.exists('trained.pckl')):
    Y = pickle.load(open('trained.pckl','rb'))
else:
    mlb = MultiLabelBinarizer()
    Y = mlb.fit_transform(y_train_text)
    pickle.dump(Y,open('trained.pckl','wb'))

In [23]:
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])

In [24]:
classifier.fit(X_train, Y)

1 loop, best of 3: 219 ms per loop


In [25]:
X_test = np.array(["It is cloudy and am depressed"])

In [26]:
predicted = classifier.predict(X_test)

In [27]:
all_labels = mlb.inverse_transform(predicted)

[('Sadness',)]


In [28]:
for item, labels in zip(X_test, all_labels):
    print('{0} => {1}'.format(item, ', '.join(labels)))

It is cloudy and am depressed => Sadness
